In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch # deep learning
from torchvision import transforms # image preprocessing
from PIL import Image # read image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

/Users/chaiharsha/Documents/streetview_predictor/streetviewenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/chaiharsha/Documents/streetview_predictor/streetviewenv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <61623A3D-DA3C-3AAD-B2F0-D363151DDB3F> /Users/chaiharsha/Documents/streetview_predictor/streetviewenv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <66FB8649-BB87-3CD6-A177-462038DCAE02> /Users/chaiharsha/Documents/streetview_predictor/streetviewenv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Get labels
df = pd.read_csv(os.path.join('coordinates.csv'))
df.head()

,latitude,longitude
0,20.824885,-98.499517
1,-3.451752,-54.563937
2,-23.496464,-47.460542
3,-16.548678,-72.852778
4,-35.010870,140.064397


In [3]:
# define the image preprocessing transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize the image
])

In [4]:
coords = []
images = []

for i, row in df.iterrows():
    coordinate = row[['latitude', 'longitude']].values.astype('float32')
    coords.append(coordinate)
    
    img_name = f"Streetview_Image_Dataset/{i}.png"  # Image filename based on 'num' column
    image = Image.open(img_name)
    image = transform(image)
    images.append(image)
    if i % 1000 == 0:
        print(f"Processed {i} images")

FileNotFoundError: [Errno 2] No such file or directory: 'Streetview_Image_Dataset/0.jpeg'

In [7]:
# Convert to torch tensors
data = torch.stack(images)
labels = torch.tensor(coords)

# Ensure the shapes are correct
print(data.shape)
print(labels.shape)


torch.Size([40229, 3, 224, 224])
torch.Size([40229, 2])


/var/folders/8v/z7_lynrn24bbpwy6p36j3cwc0000gn/T/ipykernel_59942/539367343.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  labels = torch.tensor(coords)


In [8]:
dataset = list(zip(data, labels))

# define the sizes for training, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# randomly split the dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


In [9]:
# create DataLoaders
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torch.nn as nn

# model
model = nn.Sequential(
    # 3x224x224
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.2),
    # 64x112x112
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.2),
    # 128x56x56
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    # 256x28x28
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    # 512x14x14
    nn.Flatten(),
    # 100352
    nn.Linear(100352, 4096),
    nn.ReLU(),
    nn.Dropout(0.35),
    # 4096
    nn.Linear(4096, 1024),
    nn.ReLU(),
    # 1024
    nn.Linear(1024, 256),
    nn.ReLU(),
    # 256
    nn.Linear(256, 2)
)

ideas:

make more channels
add some more conv2d layers
switch back to regular ReLU

image augmentation

image blackout

In [11]:
# loss function
criterion = nn.MSELoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# training + validation loop
num_epochs = 12
for epoch in range(num_epochs):
    # training
    model.train()
    running_train_loss = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_train_loss/len(train_loader)}")

    # validation
    model.eval()
    running_val_loss = 0.0
    
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            running_val_loss += val_loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {running_val_loss/len(val_loader)}")


Epoch 1/12, Train Loss: 4122.220004678958
Epoch 1/12, Validation Loss: 3218.5552649119545
Epoch 2/12, Train Loss: 15592497.4627654
Epoch 2/12, Validation Loss: 3517.4995775979664


KeyboardInterrupt: 

In [39]:
# save the model
torch.save(model.state_dict(), 'model1017.pth')

In [ ]:
# testing loop
model.eval()  
running_test_loss = 0.0
all_predictions = []
all_true_values = []
all_indices = []

with torch.no_grad(): 
    for batch_idx, (images, labels) in enumerate(test_loader):
        outputs = model(images)
        test_loss = criterion(outputs, labels)
        running_test_loss += test_loss.item()
        
        # predictions and true values
        all_predictions.append(outputs.cpu().numpy())  
        all_true_values.append(labels.cpu().numpy()) 
        
        # indices
        indices = test_loader.dataset.indices[batch_idx * batch_size : (batch_idx + 1) * batch_size]
        all_indices.append(indices)
        
print(f"Test Loss after training: {running_test_loss / len(test_loader)}")

# predictions
all_predictions = np.concatenate(all_predictions, axis=0)
all_true_values = np.concatenate(all_true_values, axis=0)
all_indices = np.concatenate(all_indices, axis=0)

# samples
num_samples_to_print = 5 
for i in range(num_samples_to_print):
    print(f"Sample {i + 1}: Dataset Index: {all_indices[i]}, True: {all_true_values[i]}, Predicted: {all_predictions[i]}")
